**DCGAN code for UTKFace Generation**

Using MNIST Tutorial code, GAN code with UTKFace data preprocessing

We added Conv2d layer in both D(z) and G(z) to get spatial infomation of image data.

https://github.com/yunjey/pytorch-tutorial

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 64 * 64 * 3
num_epochs = 20
batch_size = 100
sample_dir = 'samples'

In [ ]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
transform = transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
#                                      std=[0.5])])


In [ ]:
# utkface dataset

import kagglehub
download_path = kagglehub.dataset_download("jangedoo/utkface-new")
path = download_path + "/UTKFace"

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# utkface custom dataset class
class UTKFaceDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data_dir = data_dir
    self.transform = transform
    self.image_files = []
    self.labels = []

    for img_file in os.listdir(data_dir):
      if (img_file.endswith('.jpg')):
        self.image_files.append(img_file)
        try:
          age = int(img_file.split('_')[0])
          # gender = int(img_file.split('_')[1])
          # race = int(img_file.split('_')[2])
          self.labels.append(age)
        except:
          print(f"file skipped with error : {img_file}")
    # print(self.labels)

  def __len__(self):
    return len(self.image_files)

  def __getitem__(self, idx):
    img_path = os.path.join(self.data_dir, self.image_files[idx])
    image = Image.open(img_path).convert('RGB')
    label = self.labels[idx]

    if self.transform:
      image = self.transform(image)
    return image, label

utkface_dataset = UTKFaceDataset(data_dir = path, transform=transform)

utkface_dataloader = DataLoader(dataset=utkface_dataset, batch_size = batch_size, shuffle=True)

In [ ]:
# DCGAN - UTKFace
# Discriminator
D = nn.Sequential(
    nn.Conv2d(3, 64, 4, 2, 1),  # (64, 32, 32)
    nn.LeakyReLU(0.2, inplace=True),

    nn.Conv2d(64, 128, 4, 2, 1),  # (128, 16, 16)
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),

    nn.Conv2d(128, 256, 4, 2, 1),  # (256, 8, 8)
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),

    nn.Conv2d(256, 512, 4, 2, 1),  # (512, 4, 4)
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),

    nn.Conv2d(512, 1, 4, 1, 0),  # (1, 1, 1)
    nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.ConvTranspose2d(latent_size, 512, 4, 1, 0),  # (512, 4, 4)
    nn.BatchNorm2d(512),
    nn.ReLU(True),

    nn.ConvTranspose2d(512, 256, 4, 2, 1),  # (256, 8, 8)
    nn.BatchNorm2d(256),
    nn.ReLU(True),

    nn.ConvTranspose2d(256, 128, 4, 2, 1),  # (128, 16, 16)
    nn.BatchNorm2d(128),
    nn.ReLU(True),

    nn.ConvTranspose2d(128, 64, 4, 2, 1),  # (64, 32, 32)
    nn.BatchNorm2d(64),
    nn.ReLU(True),

    nn.ConvTranspose2d(64, 3, 4, 2, 1),  # (1, 64, 64)
    nn.Tanh())


# Device setting
D = D.to(device)
G = G.to(device)

In [ ]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)


def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()



In [ ]:
# Start training
total_step = len(utkface_dataloader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(utkface_dataloader):
        current_batch_size = images.size(0)
        images = images.to(device)

        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(current_batch_size, 1).to(device)
        fake_labels = torch.zeros(current_batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images).squeeze()
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(current_batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images).squeeze()
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()

        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(current_batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images).squeeze()

        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)

        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(),
                          real_score.mean().item(), fake_score.mean().item()))

    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 3, 64, 64)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))

    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 3, 64, 64)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))


In [ ]:
# Display generated images
import matplotlib.pyplot as plt

print('Generated Images')
plt.figure(figsize=(9, 9)) # Increase figure size for a 3x3 grid

# Generate and display 9 images (3 rows x 3 columns)
for k in range(9):
    plt.subplot(3, 3, k + 1)

    # create random noize vector with size(1, latent_size)
    z = torch.randn(1, latent_size).to(device)
    gen_img = G(z).detach().cpu() # give noise vector to generator and create fake image

    gen_img = gen_img.reshape(3, 64, 64) # reshaping flatten to image tensor
    gen_img = denorm(gen_img) # denormalize

    gen_img = gen_img.permute(1, 2, 0)

    plt.imshow(gen_img.numpy())
    plt.axis('off')

plt.tight_layout()
plt.show()